<a href="https://colab.research.google.com/github/Prasad-Mamidi/Machine_Learning/blob/main/Product_Classiffication_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd

In [2]:
prod_train = pd.read_csv('https://raw.githubusercontent.com/Prasad-Mamidi/Machine_Learning/main/Datasets/Ecom_Product/train.csv')
prod_test = pd.read_csv('https://raw.githubusercontent.com/Prasad-Mamidi/Machine_Learning/main/Datasets/Ecom_Product/test.csv')

In [3]:
prod_train.head()

,id,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,...,spec92,spec93,spec94,spec95,spec96,spec97,spec98,spec99,spec100,Category
0,1,1,0,0,0,0,0,0,0,0,...,8,9,0,4,0,12,0,0,0,Mobiles
1,2,0,0,0,0,0,0,0,1,0,...,7,1,0,0,0,1,0,0,0,Mobiles
2,3,0,0,0,0,0,0,0,1,0,...,7,9,0,0,0,9,0,0,0,Mobiles
3,4,1,0,0,1,6,1,5,0,0,...,7,14,0,15,0,21,0,0,0,Mobiles
4,5,0,0,0,0,0,0,0,0,0,...,10,7,0,0,0,12,0,0,0,Mobiles


In [4]:
prod_train.columns

Index(['id', 'spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6', 'spec7',
       'spec8', 'spec9',
       ...
       'spec92', 'spec93', 'spec94', 'spec95', 'spec96', 'spec97', 'spec98',
       'spec99', 'spec100', 'Category'],
      dtype='object', length=102)

In [5]:
print(prod_train["Category"].value_counts())
print(len(prod_train["Category"].unique()))

Category
Tablets          13079
Personal_Care    11452
Appliances        6863
Laptops           6466
Camara            3964
Accessories       2341
Ipod              2217
TV                2182
Mobiles           1558
Name: count, dtype: int64
9


##prepare train,test data

In [6]:
X_train = prod_train.drop(["Category"],axis=1)
y_train = prod_train["Category"]

X_test = prod_test.drop(["Category"],axis=1)
y_test = prod_test["Category"]

##DecisionTreeClassifier

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
dct   = DecisionTreeClassifier(max_depth=4)
dct = dct.fit(X_train,y_train)

In [8]:
#Train Accuracy score
print(accuracy_score(y_train,dct.predict(X_train)))
# print(dct.score(X_train,y_train))
#Test Accuracy score
print(accuracy_score(y_test,dct.predict(X_test)))
# print(dct.score(X_test,y_test))


0.9097601851482383
0.9142565498468866


##GradientBoostingAlgorithm

In [9]:
import time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,f1_score

In [10]:


gbc = GradientBoostingClassifier(n_estimators=50,max_depth=1,learning_rate=0.01)
gbc = gbc.fit(X_train,y_train)

start_time = time.time()
print("Time Taken by GBC "+str(time.time()-start_time)+"Seconds")
# print("Accuracy of GBC "+str(gbc.score(X_test,y_test)))

Time Taken by GBC 7.128715515136719e-05Seconds


In [11]:
print(confusion_matrix(y_train,gbc.predict(X_train)))
print("Train Accuracy of GBC "+str(gbc.score(X_train,y_train)))
print(confusion_matrix(y_test,gbc.predict(X_test)))
print("Test Accuracy of GBC "+str(gbc.score(X_test,y_test)))

[[  545     0     0    23     0     0  1773     0     0]
 [    0  6661     0     4     0     0   198     0     0]
 [    0     0  3964     0     0     0     0     0     0]
 [    0     0     0  1944     0     0   273     0     0]
 [    0     0     0    97  6357     0    12     0     0]
 [    0     0     0     0     0     0     0     0  1558]
 [    0     0     0    18     0     0 11434     0     0]
 [    0     0     0    14     0     0  2168     0     0]
 [    0     0     0     0     0     0     0     0 13079]]
Train Accuracy of GBC 0.8775388053150314
[[ 106    0    0   13    0    0  379    0    0]
 [   0 1565    0    0    0    0   36    0    0]
 [   0    1  990    0    0    0    0    0    0]
 [   0    0    0  450    0    0   72    0    0]
 [   0    0    0   19 1519    0    0    0    0]
 [   0    0    0    0    0    0    0    0  371]
 [   0    0    0    8    0    0 2675    0    0]
 [   0    0    0    4    1    0  504    0    0]
 [   0    0    0    0    0    0    0    0 3043]]
Test Accurac

#XGB Alogrithm


In [20]:
import xgboost
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [16]:
#Converting Categorical Values into Numeric Form
train_labels = LabelEncoder().fit_transform(y_train.values)
test_labels = LabelEncoder().fit_transform(y_test.values)

In [19]:
y_train.values

array(['Mobiles', 'Mobiles', 'Mobiles', ..., 'Camara', 'Camara', 'Camara'],
      dtype=object)

In [17]:
train_labels

array([5, 5, 5, ..., 2, 2, 2])

In [50]:
matrix_train  = xgboost.DMatrix(X_train,label=train_labels)
matrix_test = xgboost.DMatrix(X_test,label=test_labels)

params = {
            "max_depth":1,
            "eta":0.01,
            "eval_metric":'merror',
            "tree_method":"hist",
            "device":"cuda",
            "num_class":9
         }

start_time = time.time()
model = xgboost.train(params=params,
                      dtrain=matrix_train,
                      num_boost_round=3000,
                      evals=[(matrix_test,"test")],
                      early_stopping_rounds=4)

print("Time Taken by XGB "+str(time.time()-start_time)+"Seconds")

[0]	test-merror:0.09433
[1]	test-merror:0.09433
[2]	test-merror:0.09263
[3]	test-merror:0.09323
[4]	test-merror:0.09323
[5]	test-merror:0.09323
[6]	test-merror:0.09221
[7]	test-merror:0.09221
[8]	test-merror:0.09297
[9]	test-merror:0.09297
[10]	test-merror:0.04917
[11]	test-merror:0.04917
[12]	test-merror:0.04917
[13]	test-merror:0.04917
Time Taken by XGB 0.2809865474700928Seconds


##XGB results

In [51]:
print(confusion_matrix(train_labels,model.predict(matrix_train)))
print("Train Accuracy of XGB "+str(accuracy_score(train_labels,model.predict(matrix_train))))
print(confusion_matrix(test_labels,model.predict(matrix_test)))
print("Test Accuracy of XGB "+str(accuracy_score(test_labels,model.predict(matrix_test))))

[[  663   102     0    25     0     0  1551     0     0]
 [    2  6652     0    11     0     0   198     0     0]
 [    0    47  3917     0     0     0     0     0     0]
 [    0     0     0  2032     0     0   185     0     0]
 [   14     0     0   143  6250     0    12     0    47]
 [    0     0     0     0     0  1558     0     0     0]
 [  134     0     0    54     0     0 11264     0     0]
 [    1     0     0    20    39     0     0  2122     0]
 [    0     0     0     0     0     9     0     0 13070]]
Train Accuracy of XGB 0.9482462790790471
[[ 129   20    0   13    0    0  336    0    0]
 [   0 1561    0    4    0    0   36    0    0]
 [   0    7  984    0    0    0    0    0    0]
 [   0    0    0  469    0    0   53    0    0]
 [   2    0    0   28 1502    0    0    0    6]
 [   0    0    0    0    0  371    0    0    0]
 [  32    0    0   17    0    0 2634    0    0]
 [   0    0    0    5    9    0    0  495    0]
 [   0    0    0    0    0   10    0    0 3033]]
Test Accurac